<a href="https://colab.research.google.com/github/tjfdk0410/CS489_covid19_detection_web_extension/blob/master/FN_Detection_Failver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install https://github.com/ufoym/imbalanced-dataset-sampler/archive/master.zip


     - 297 kB 1.7 MB/s
  Created wheel for torchsampler: filename=torchsampler-0.1.1-py3-none-any.whl size=3839 sha256=5d8b5e84db3beb802f61a23afbf2d3e4821ee6c90215cd7ab6da44fc5de3396c
  Stored in directory: /tmp/pip-ephem-wheel-cache-540hzcbj/wheels/52/7b/7d/ce0e0ddbb7864877a0e31a96f883a928791ebfa6eaf7b52f87
Successfully built torchsampler


In [ ]:
!pip install datasets
!pip install transformers
import torch
import transformers
from datasets import load_dataset
from torch.utils.data import DataLoader
import numpy as np
from datasets import load_dataset
from datasets import load_metric
from transformers import AutoTokenizer, training_args
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

from torchsampler import ImbalancedDatasetSampler

def get_data_loaders(
    tokenizer: transformers.PreTrainedTokenizer,
    batch_size: int = 32,
    return_loader: bool = True,
    use_imbalanced: bool = True,
    device = "cpu"
):
  if use_imbalanced and device == "cuda":
    raise ValueError("use_imbalance == True에 device = 'cuda'이면 오류가 발생합니다. use_imbalanced를 False로 하거나 device를 cpu로 하십시오.")
    
  device = torch.device(device)

all_data = load_dataset(
  "csv",
  data_files={
    "train": "/train2.csv",
    "val": "/val.csv",
    "test": "/test.csv"
    },
  )


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_fuction(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = all_data.map(tokenize_fuction, batched=True)

"""small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))"""
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

print(full_train_dataset)
print(type(full_train_dataset))


model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
epoch =20

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

trainer = Trainer(model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset)

trainer.train
print(trainer)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis = -1)
    return metric.compute(predictions=predictions, references=labels)
epoch =20
trainin_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")
trainer = Trainer(model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset, compute_metrics=compute_metrics)

t = trainer.evaluate()
epoch = 20
training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

print(t)

Using custom data configuration default-d6de76700400c7de


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d6de76700400c7de/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 200
  Batch size = 8


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


{'eval_loss': 0.7184663414955139, 'eval_accuracy': 0.425, 'eval_runtime': 15.1377, 'eval_samples_per_second': 13.212, 'eval_steps_per_second': 1.652}
